## Постановка задачи
* Кластеризовать регионы России и определить, какие из них наиболее остро нуждаются в помощи малообеспеченным/неблагополучным слоям населения;
* Описать группы населения, сталкивающиеся с бедностью;
* Определить: 
   * влияет ли число детей, пенсионеров и других социально уязвимых групп на уровень бедности в регионе;
   * связаны ли уровень бедности/социального неблагополучия с производством и потреблением в регионе; 
   * какие ещё зависимости можно наблюдать относительно социально незащищённых слоёв населения.

Вывод: ... Однако, полученную модеть обучения следует провести на новых данных, так как в 2022 году изменилась внешнеполитическая ситуация связанная с СВО, ужесточилась политика ЦБ РФ, ведущая к падению производства, но, в то же время увеличилась поддержка малоимущих слоёв населения, изменились потоки миграции.

## ⭐️ 1. Знакомство с данными и первичный анализ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import re
import pickle

In [2]:
from IPython.display import display, HTML

#warnings.filterwarnings("ignore")

plt.rcParams["patch.force_edgecolor"] = True

### Обзор данных
Для проекта предоставлены следующие данные:
- child_mortality_rural_1990_2021.xls — число умерших на первом году жизни детей за год, по всем регионам, в сельской местности;
- child_mortality_urban_1990_2021.xls — число умерших на первом году жизни детей за год, по всем регионам, в городской местности;
- disabled_total_by_age_2017_2022.csv — число людей с инвалидностью по регионам, по месяцам, по возрастным группам;
- morbidity_2005_2020_age_disease.xls — заболеваемость на 100тыс. человек населения,по возрастным группам и группам заболеваний;
- poverty_percent_by_regions_1992_2020.csv — процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам;
- welfare_expense_share_2015_2020 — расходы на социальную политику от общих расходов бюджета региона, % в год;
- cash_real_income_wages_2015_2020 — среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам;
- poverty_socdem_20*.xls — распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020гг., по регионам;
- housing_2020 — характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование2020года;
- population.xlsx — численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022гг. 
- gross_regional_product_1996_2020.xls — валовой региональный продукт на душу населения, в рублях;
- regional_production_*_*.csv — объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности за 2005–2016 гг., 2017–2020гг.(в тысячах рублей, значение показателя за год,полный круг);
- retail_turnover_per_capita_2000_2021.xls — оборот розничной торговли на душу населения, в рублях;
- crimes — сведения о преступлениях, совершённых отдельными категориями лиц за 2016–2022гг., по месяцам, регионам, категориям лиц, категориям преступлений;
- drug_alco — сведения о заболеваемости алкоголизмом и наркоманией, на 100 тыс.населения(2005–2018);
- newborn_2006_2022_monthly.csv — рождённые в этом месяце, по регионам, без учёта мертворождённых;
- workers.csv — отношение числа занятых в экономике региона к численности населения региона в трудоспособном возрасте, %, 2012–2020гг.

Следует отметить, что для более объективных выводов были найдены дополнительные данные из официальных статистических источников:
- сhislo_bolnich_organiz.xlsx - количества больниц \
https://rosstat.gov.ru/folder/13721
- scool_building_2018.xlsx - количество зданий школ \
https://www.fedstat.ru/indicator/59606
-  credit_20*.xlsx - сведения о задолженности физлиц перед кредитными организациями \
https://www.fedstat.ru/indicator/42861
- migr17.xls - общий миграционный прирост \
https://rosstat.gov.ru/bgd/regl/b17_107/Main.htm
- job.xls - количество безработных в России по регионам,тыс. чел. \
https://gogov.ru/articles/unemployment-rate

Показатель граждан ставших на учёт у нарколога считаю необъективным, поэтому дополнительно будет рассмотрен показатель розничной торговли алкогольной продукции
- data_alko_cach.xlsx \
https://www.fedstat.ru/indicator/59721

Данные о криминальной ситуации в данном проекте рассматриваться не будут, так как они являются отражением и следствием социально демографического положения.



### Преобразование данных
Для построения итогового датасета, все данные будут приведены к показателю на душу населения, проживающего в регионе или же усреднённые за период 2017-2020 годы.

Для преобразования наименований регионов в стандартизированную форму используем функцию federal_subj()

- df - DataFrame;
- standard_names - словарь стандартных названий; 
- список names_to_drop позволяет сразу исключить некоторые регионы, которые могут быть ошибочно обработаны функцией как федеральные субъекты;
- region_col - колонка DataFrame с названиями регионов.

In [3]:
def federal_subj(
    df, standard_names, names_to_drop=None, region_col='region'):
    '''Приведение наименований регионов к единому виду для разных таблиц
    args Таблицы статистических данных, таблица стандартных наименований регионов
    '''
       
    def get_standard_name(x):
        '''Просмотр столбца регионов и вывод названий в стандартном виде'''
    
        for key, value in standard_names.items():
            x = re.sub('[\(].*?[\)]', '', x)  # удаляем лишние знаки
            if re.search(key, x.lower()):    #приводим к нижнему регистру
                return value
        return None
    
    if names_to_drop is None:
        names_to_drop = []
    df = df[~df[region_col].isin(names_to_drop)].copy()   #выбираем только нужные строки
    df[region_col] = df[region_col].apply(get_standard_name) #применяем к столбцу региона функцию стандартизации названий
    return (df.dropna(subset=region_col).set_index(region_col).sort_index()) #удаляем строки с пропущенными значениями и обновляем индексы

### 1.1 Доходы населения
#### Чтение данных

| Dataframe | Описание |
| - | - |
| per_capita: | Среднедушевые денежные доходы (в месяц), руб. |
| real_incomes | Реальные денежные доходы, в процентах к предыдущему году (покупательная способность: с учетом роста цен)|
| formal_wage |Среднемесячная номинальная начисленная заработная плата, руб. |
| real_pay | Реальная начисленная заработная плата, в процентах к предыдущему году (покупательная способность: с учетом роста цен)|

In [4]:
def finances_readout(file, sheet):
    ''' Получение показателей среднедушевого дохода, руб.
    args функция получает табличный файл и номер листа в ней
    Отдаёт отдельные таблицы по каждому листу
    '''
        
    df = pd.read_excel(file, sheet_name=sheet)   #чтение файла
    names_to_drop = ['Архангельская область', 'Тюменская область'] #удаление дублирующихся регионов
    df = federal_subj(df, standard_names, names_to_drop)   #Применение функции стандартизации наименований регионов
    df.columns = df.columns.astype(object) 
    return df
    
with open('social_russia_data/standard_names.pkl', 'rb') as f:  #Загрузка файла со стандартными наименованиями
    standard_names = pickle.load(f)
    
    
file = './social_russia_data/cash_real_income_wages_2015_2020.xlsx'
per_capita = finances_readout(file, 0)
real_incomes = finances_readout(file, 1)
formal_wage = finances_readout(file, 2)
real_pay = finances_readout(file, 3)


In [5]:
display('Среднедушевой доход, руб.мес./чел.', per_capita.head(3))
display('Реальный доход, %', real_incomes.head(3))
display('Номинальная зарплата, руб.', formal_wage.head(3))
display('Реальная зарплата, %', real_pay.head(3))

'Среднедушевой доход, руб.мес./чел.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20860.0,21256.0,22139.0,22829.0,23937.0,23864.0
Амурская область,28240.0,27976.0,29213.0,30937.0,33304.0,35499.0
Архангельская область без Ненецкого АО,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0


'Реальный доход, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,99.1,94.7,100.0,99.7,99.6,95.5
Амурская область,96.1,92.1,101.1,102.4,101.7,100.3
Архангельская область без Ненецкого АО,95.1,93.0,98.7,101.7,100.2,98.4


'Номинальная зарплата, руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20090.0,21202.0,22743.0,25519.0,27962.0,30072.0
Амурская область,32902.0,33837.0,37368.0,42315.0,47234.0,52430.0
Архангельская область без Ненецкого АО,35592.0,38118.0,40352.0,45427.0,49435.0,52779.0


'Реальная зарплата, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,90,98.4,103.6,109.3,104.9,103.8
Амурская область,88,96.0,107.4,110.1,106.0,105.2
Архангельская область без Ненецкого АО,92.4,100.0,102.5,110.7,104.0,102.9


In [6]:
real_pay.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2015    85 non-null     object 
 1   2016    85 non-null     float64
 2   2017    85 non-null     float64
 3   2018    85 non-null     float64
 4   2019    85 non-null     float64
 5   2020    85 non-null     float64
dtypes: float64(5), object(1)
memory usage: 4.6+ KB


#### Средние значения на душу населения, руб; %

In [7]:
per_capita_mean = per_capita.iloc[:, 2:].mean(axis=1)
per_capita_mean_df = per_capita_mean.to_frame (name='Среднедушевой доход, руб.мес./чел.')
per_capita_mean_df = per_capita_mean_df.reset_index()

In [8]:
real_incomes_mean = real_incomes.iloc[:, 2:].mean(axis=1)
real_incomes_mean_df = real_incomes_mean.to_frame (name='Реальные доходы,руб.мес./чел.')
real_incomes_mean_df = real_incomes_mean_df.reset_index()

In [9]:
formal_wage_mean = formal_wage.iloc[:, 2:].mean(axis=1)
formal_wage_mean_df = formal_wage_mean.to_frame (name='Начисленная з/п,руб.мес./чел.')
formal_wage_mean_df = formal_wage_mean_df.reset_index()

In [10]:
real_pay_mean = real_pay.iloc[:, 2:].mean(axis=1)    
real_pay_mean_df = real_pay_mean.to_frame (name='Реальная з/п, % к прошлому году/чел.')
real_pay_mean_df = real_pay_mean_df.reset_index()

### 1.2 Численность постоянного населения, человек

In [11]:
population = pd.read_excel('social_russia_data/population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)

names_to_drop = [
        'Архангельская область (кроме Ненецкого автономного округа)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
        'Муниципальные образования Алтайского края',
        'Алтайский муниципальный район',
        'Алтайский сельсовет',
        'Большекалтайский сельсовет',
        'Населенные пункты, входящие в состав муниципальных образований Алтайского края',
        'Кировский сельсовет'
]
population = federal_subj(population, standard_names, names_to_drop)
population.loc['Архангельская область без Ненецкого АО'] \
    = (population.loc['Архангельская область без Ненецкого АО']
      - population.loc['Ненецкий автономный округ'])
population.loc['Тюменская область без округов'] \
    = (population.loc['Тюменская область без округов']
       - population.loc['Ханты-Мансийский автономный округ - Югра']
       - population.loc['Ямало-Ненецкий автономный округ'])

newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = population.columns.astype(object)     
population = population.rename(columns={     #переименовываем столбцы
    'январь 1999 г.': '1999', 
    'январь 2000 г.': '2000',
    'январь 2001 г.': '2001',
    'январь 2002 г.': '2002',
    'январь 2003 г.': '2003',
    'январь 2004 г.': '2004',
    'январь 2005 г.': '2005',
    'январь 2006 г.': '2006',
    'январь 2007 г.': '2007',
    'январь 2008 г.': '2008',
    'январь 2009 г.': '2009',
    'январь 2010 г.': '2010',
    'январь 2011 г.': '2011',
    'январь 2012 г.': '2012',
    'январь 2013 г.': '2013',
    'январь 2014 г.': '2014',
    'январь 2015 г.': '2015',
    'январь 2016 г.': '2016',
    'январь 2017 г.': '2017',
    'январь 2018 г.': '2018',
    'январь 2019 г.': '2019',
    'январь 2020 г.': '2020',
    'январь 2021 г.': '2021',
    'январь 2022 г.': '2022'
    })   

In [12]:
population.head(3)

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2662738.0,2651628.0,2641079.0,2621050.0,2602595.0,2571987.0,2539430.0,2503510.0,2473024.0,2453455.0,...,2398751.0,2390638.0,2384812.0,2376774.0,2365680.0,2350080.0,2332813.0,2317153.0,2296353.0,2268179.0
Амурская область,949526.0,935607.0,923055.0,911381.0,901044.0,887781.0,874018.0,861056.0,850502.0,844290.0,...,816910.0,811274.0,809873.0,805689.0,801752.0,798424.0,793194.0,790044.0,781846.0,772525.0
Архангельская область без Ненецкого АО,1372631.0,1349160.0,1328187.0,1309552.0,1290956.0,1273668.0,1257312.0,1239924.0,1224813.0,1215264.0,...,1159506.0,1148760.0,1139950.0,1130240.0,1121813.0,1111031.0,1100290.0,1092424.0,1082662.0,1069782.0


In [13]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1999    83 non-null     float64
 1   2000    83 non-null     float64
 2   2001    83 non-null     float64
 3   2002    83 non-null     float64
 4   2003    83 non-null     float64
 5   2004    83 non-null     float64
 6   2005    83 non-null     float64
 7   2006    83 non-null     float64
 8   2007    83 non-null     float64
 9   2008    83 non-null     float64
 10  2009    83 non-null     float64
 11  2010    83 non-null     float64
 12  2011    83 non-null     float64
 13  2012    83 non-null     float64
 14  2013    83 non-null     float64
 15  2014    83 non-null     float64
 16  2015    85 non-null     float64
 17  2016    85 non-null     float64
 18  2017    85 non-null     float64
 19  2018    85 non-null     float64
 20  2019    85 non-null     float64
 21  2020    85 non-n

Расчитаем среднюю численность населения за период 2017-2020 годы.

In [14]:
population_mean = population.iloc[:, 18:22].mean(axis=1)
population_mean_df = population_mean.to_frame (name='Численность населения, чел.')
population_mean_df = population_mean_df.reset_index()

### 1.3 Сведения о задолженности по кредитам,  предоставленным  кредитными организациями физическим лицам-резидентам (миллион рублей, в рублях)
Данные получены из официальных статистических источников. Единица измерения - миллион рублей.\
https://www.fedstat.ru/indicator/59345

#### Для данных 2017 года проведём предварительную обработку.

Очистка данных:
- удаление неинформативных столбцов;
- переименовывание столбцов;
- удаление первых двух строк; 
- изменение типа данных на числовой;
- сложение месячных показателей в годовой итог; 
- сортировка регионов по алфавиту.

In [15]:
df_kredit_2017 = pd.read_excel('dop_social_data\credit_2017.xls')

def std_data(df):
    '''Функция удаляет неинформативные столбцы и переименовывает их, удаляет первые две строки, 
    изменяет тип данных на числовой, складывает месячные показатели в годовой итог, сортирует регионы по алфавиту
    '''
    
    df = df.drop([
                'Сведения о задолженности по кредитам,  предоставленным  кредитными организациями физическим лицам-резидентам (миллион рублей, в рублях)',
                'Unnamed: 2',
                'Unnamed: 3'],
                axis=1
                )  
    df = df.rename(columns={'Unnamed: 1': 'region', 'Unnamed: 4': 'Задолженность'})    #переименовываем столбцы
    df = df.drop(index=[ 0 , 1]) #удаляем строки  
    df.reset_index(drop= True , inplace= True )    #сброс индекса
    df['Задолженность'] = df['Задолженность'].astype(int)    #смена типа данных
    df = df.groupby(['region'])['Задолженность'].sum().reset_index()   #группировка по годам
    return df


kredit_2017 = std_data(df_kredit_2017)
kredit_2017.head()

,region,Задолженность
0,Архангельская область без данных п...,1200852
1,Тюменская область без данных по Ха...,2010344
2,в том числе Ненецкий автономный округ,66263
3,в том числе Ханты-Мансийский автон...,3575735
4,в том числе Ямало-Ненецкий автоном...,1286509


In [16]:
kredit_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   region         84 non-null     object
 1   Задолженность  84 non-null     int32 
dtypes: int32(1), object(1)
memory usage: 1.1+ KB


В таблице отсутствуют значения по Чукотскому автономному округу. Добавим строку с нулевыми значениями.

In [17]:
dop_region = {'region':'Чукотский автономный округ', 'Задолженность':0}
data_kredit_2017 = kredit_2017.append(dop_region, ignore_index=True)
data_kredit_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   region         85 non-null     object
 1   Задолженность  85 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


C:\Users\Трен\AppData\Local\Temp\ipykernel_5824\1000517620.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_kredit_2017 = kredit_2017.append(dop_region, ignore_index=True)


In [18]:
data_kredit_2017.head(3)

,region,Задолженность
0,Архангельская область без данных п...,1200852
1,Тюменская область без данных по Ха...,2010344
2,в том числе Ненецкий автономный округ,66263


Приведение наименований регионов к стандартному типу как в предыдущей таблице о доходах.

In [19]:
def federal_subj_modify(
    df, standard_names, names_to_drop=None, region_col='region'):
    '''Модифицированная функция для приведения наименований регионов к единому виду для разных таблиц'''
       
    def get_standard_name(x):
        '''Просмотр столбца регионов и вывод названий в стандартном виде'''
    
        for key, value in standard_names.items():
            x = re.sub('[\(].*?[\)]', '', x)  # удаляем лишние знаки
            if re.search(key, x.lower()):    #приводим к нижнему регистру
                return value
        return None
    
    if names_to_drop is None:
        names_to_drop = []
    df = df[~df[region_col].isin(names_to_drop)].copy()   #выбираем только нужные строки
    df[region_col] = df[region_col].apply(get_standard_name) #применяем к столбцу региона функцию стандартизации названий
    return (df.dropna(subset=region_col).set_index(region_col).sort_index()) #удаляем строки с пропущенными значениями и обновляем индексы

In [20]:
def kredit_readout(df):
    ''' Приведение наименований регионов к единому виду'''
           
    names_to_drop = ['Архангельская область', 'Тюменская область'] #удаление дублирующихся регионов
    df = federal_subj_modify(df, standard_names, names_to_drop)   #Применение функции стандартизации наименований регионов
    return df
    
with open('social_russia_data/standard_names.pkl', 'rb') as f:  #Загрузка файла со стандартными наименованиями
    standard_names = pickle.load(f)
    
#data_kredit_2017.columns = data_kredit_2017.columns.astype(object)     
data_kredit_2017 = kredit_readout(data_kredit_2017)
data_kredit_2017.head(3)

,Задолженность
region,
Алтайский край,1725755
Амурская область,832725
Архангельская область без Ненецкого АО,1200852


In [21]:
data_kredit_2017 = data_kredit_2017.rename(columns={'Задолженность': '2017'})
data_kredit_2017.head()

,2017
region,
Алтайский край,1725755
Амурская область,832725
Архангельская область без Ненецкого АО,1200852
Астраханская область,853813
Белгородская область,1156580


#### Аналогично подготовим данные для 2018-2020 годов

Закредитованность в 2018 году.

In [22]:
df_kredit_2018 = pd.read_excel('dop_social_data\credit_2018.xls')
kredit_2018 = std_data(df_kredit_2018)
data_kredit_2018 = kredit_2018.append(dop_region, ignore_index=True)
data_kredit_2018.columns = data_kredit_2018.columns.astype(object)     
data_kredit_2018 = kredit_readout(data_kredit_2018)
data_kredit_2018 = data_kredit_2018.rename(columns={'Задолженность': '2018'})

data_kredit_2018.head()

C:\Users\Трен\AppData\Local\Temp\ipykernel_5824\1727904702.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_kredit_2018 = kredit_2018.append(dop_region, ignore_index=True)


,2018
region,
Алтайский край,2007501
Амурская область,949383
Архангельская область без Ненецкого АО,1409440
Астраханская область,980762
Белгородская область,1350146


Закредитованность в 2019 году.

In [23]:
df_kredit_2019 = pd.read_excel('dop_social_data\credit_2019.xls')
kredit_2019 = std_data(df_kredit_2019)
data_kredit_2019 = kredit_2019.append(dop_region, ignore_index=True)
data_kredit_2019.columns = data_kredit_2019.columns.astype(object)     
data_kredit_2019 = kredit_readout(data_kredit_2019)
data_kredit_2019 = data_kredit_2019.rename(columns={'Задолженность': '2019'})
data_kredit_2019.head()

C:\Users\Трен\AppData\Local\Temp\ipykernel_5824\798124523.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_kredit_2019 = kredit_2019.append(dop_region, ignore_index=True)


,2019
region,
Алтайский край,2386761
Амурская область,1128514
Архангельская область без Ненецкого АО,1668797
Астраханская область,1154601
Белгородская область,1632737


Закредитованность в 2020 году.

In [24]:
df_kredit_2020 = pd.read_excel('dop_social_data\credit_2020.xls')
kredit_2020 = std_data(df_kredit_2020)
data_kredit_2020 = kredit_2020.append(dop_region, ignore_index=True)
data_kredit_2020.columns = data_kredit_2020.columns.astype(object)     
data_kredit_2020 = kredit_readout(data_kredit_2020)
data_kredit_2020 = data_kredit_2020.rename(columns={'Задолженность': '2020'})
data_kredit_2020.head()

C:\Users\Трен\AppData\Local\Temp\ipykernel_5824\3149061243.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_kredit_2020 = kredit_2020.append(dop_region, ignore_index=True)


,2020
region,
Алтайский край,2752473
Амурская область,1302909
Архангельская область без Ненецкого АО,1883676
Астраханская область,1307144
Белгородская область,1871287


#### Обьединяем таблицы в сводную по всем годам

In [25]:
kredit_joined = pd.concat([data_kredit_2017, data_kredit_2018, data_kredit_2019, data_kredit_2020], axis=1)
kredit_joined.columns = kredit_joined.columns.str.replace(' ', '')    #Убираем пробелы в названиях столбцов
kredit_joined.head()

,2017,2018,2019,2020
region,,,,
Алтайский край,1725755,2007501,2386761,2752473
Амурская область,832725,949383,1128514,1302909
Архангельская область без Ненецкого АО,1200852,1409440,1668797,1883676
Астраханская область,853813,980762,1154601,1307144
Белгородская область,1156580,1350146,1632737,1871287


In [26]:
kredit_joined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2017    85 non-null     int64
 1   2018    85 non-null     int64
 2   2019    85 non-null     int64
 3   2020    85 non-null     int64
dtypes: int64(4)
memory usage: 3.3+ KB


#### Задолженность по кредитам на душу населения, млн. руб./чел.

In [27]:
kredit_joined_mean = kredit_joined.mean(axis=1)
kredit_individ = kredit_joined_mean/population_mean
kredit_individ_df = kredit_individ.to_frame(name='Кредитная задолженность, млн.руб./чел.')   
kredit_individ_df = kredit_individ_df.reset_index()
kredit_individ_df.head()

,region,"Кредитная задолженность, млн.руб./чел."
0,Алтайский край,0.947336
1,Амурская область,1.323589
2,Архангельская область без Ненецкого АО,1.392540
3,Астраханская область,1.059191
4,Белгородская область,0.969584


### 1.4 Число умерших на первом году жизни детей за год, по всем регионам
Единица измерения - количество человек. Берем суммарные данные по городской и сельской статистике. Данные по Архангельской и Тюменской областям пересчитываем самостоятельно из-за различающейся организации данных в зависимости от года.

In [28]:
def mortality_readout(file):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, header=2, skipfooter=13, usecols='A, D:AI')
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    names_to_drop = [
        '            Архангельская область (кроме Ненецкого автономного округа)',
        '            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    ]
    df = federal_subj(df, standard_names, names_to_drop)
    df.loc['Архангельская область без Ненецкого АО'] \
        = (df.loc['Архангельская область без Ненецкого АО']
           - df.loc['Ненецкий автономный округ'])
    df.loc['Тюменская область без округов'] \
        = (df.loc['Тюменская область без округов']
           - df.loc['Ханты-Мансийский автономный округ - Югра']
           - df.loc['Ямало-Ненецкий автономный округ'])
    df.columns = df.columns.astype(object) 
    return df


rural = mortality_readout('social_russia_data/child_mortality_rural_1990_2021.xls')
urban = mortality_readout('social_russia_data/child_mortality_urban_1990_2021.xls')
child_mortality = rural.add(urban)    #объединяем таблицы

child_mortality.columns = child_mortality.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
child_mortality.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,522.0,567.0,529.0,560.0,497.0,490.0,405.0,375.0,340.0,391.0,...,330.0,290.0,275.0,222.0,211.0,185.0,176.0,114.0,97.0,88.0
Амурская область,343.0,293.0,280.0,244.0,242.0,250.0,245.0,268.0,246.0,270.0,...,161.0,119.0,101.0,83.0,53.0,48.0,46.0,43.0,42.0,36.0
Архангельская область без Ненецкого АО,288.0,274.0,245.0,237.0,224.0,206.0,194.0,157.0,167.0,168.0,...,108.0,113.0,97.0,84.0,79.0,72.0,53.0,58.0,29.0,32.0
Астраханская область,250.0,235.0,188.0,216.0,202.0,194.0,192.0,188.0,221.0,154.0,...,142.0,130.0,148.0,119.0,82.0,74.0,71.0,72.0,64.0,46.0
Белгородская область,312.0,290.0,240.0,244.0,233.0,203.0,174.0,167.0,160.0,161.0,...,127.0,116.0,103.0,110.0,108.0,77.0,73.0,39.0,47.0,54.0


In [29]:
child_mortality.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1990    83 non-null     float64
 1   1991    83 non-null     float64
 2   1992    83 non-null     float64
 3   1993    83 non-null     float64
 4   1994    83 non-null     float64
 5   1995    83 non-null     float64
 6   1996    83 non-null     float64
 7   1997    80 non-null     float64
 8   1998    80 non-null     float64
 9   1999    80 non-null     float64
 10  2000    81 non-null     float64
 11  2001    81 non-null     float64
 12  2002    81 non-null     float64
 13  2003    81 non-null     float64
 14  2004    81 non-null     float64
 15  2005    81 non-null     float64
 16  2006    81 non-null     float64
 17  2007    81 non-null     float64
 18  2008    81 non-null     float64
 19  2009    81 non-null     float64
 20  2010    81 non-null     float64
 21  2011    81 non-n

#### Число умерших на первом году жизни детей за год, по всем регионам, чел./1000чел.

In [30]:
child_mortality_mean = child_mortality.iloc[:, 27:30].mean(axis=1) #смертность, всего
child_mortality_individ = 1000 * child_mortality_mean / population_mean
child_mortality_individ_df = child_mortality_individ.to_frame(name='Младенческая смертность, на тыс.чел.') 
child_mortality_individ_df = child_mortality_individ_df.reset_index()

### 1.5 Количество безработных в России по регионам,тыс. чел.

In [31]:
job_df = pd.read_csv('dop_social_data\job.csv', 
                     converters={'2000': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2001': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2002': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2003': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2004': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2005': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2006': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2007': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2008': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2009': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2010': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2011': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2012': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2013': lambda x: pd.to_numeric(x, errors='coerce'),
                               '2014': lambda x: pd.to_numeric(x, errors='coerce')
                              })

job_df = job_df.rename(columns={'region': 'region'})
job_df.columns = job_df.columns.astype(object) 
job_df.columns = job_df.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
job_df = job_df.rename(columns={'регион': 'region'})
job_df = kredit_readout(job_df)

job_df.head(3)

,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,...,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,60.6,65.2,66.1,70.3,78.2,99.6,94.8,83.4,95.8,74.4,...,152.0,105.1,81.7,114.4,112.9,117.6,155.3,106.1,123.7,149.0
Амурская область,20.7,24.0,21.9,22.8,24.3,24.6,24.0,23.5,25.7,22.7,...,36.7,23.2,26.1,35.3,41.4,49.8,45.0,48.4,54.5,66.1
Архангельская область без Ненецкого АО,35.0,38.3,33.2,35.1,36.2,40.2,40.0,43.1,36.6,32.6,...,45.9,43.7,35.2,39.4,35.4,47.0,68.3,56.4,59.5,87.4


In [32]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, Алтайский край to Ярославская область
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2021    84 non-null     float64
 1   2020    84 non-null     float64
 2   2019    84 non-null     float64
 3   2018    84 non-null     float64
 4   2017    84 non-null     float64
 5   2016    84 non-null     float64
 6   2015    84 non-null     float64
 7   2014    82 non-null     float64
 8   2013    82 non-null     float64
 9   2012    82 non-null     float64
 10  2011    82 non-null     float64
 11  2010    82 non-null     float64
 12  2009    82 non-null     float64
 13  2008    82 non-null     float64
 14  2007    82 non-null     float64
 15  2006    82 non-null     float64
 16  2005    82 non-null     float64
 17  2004    82 non-null     float64
 18  2003    82 non-null     float64
 19  2002    82 non-null     float64
 20  2001    82 non-null     float64
 21  2000    82 non-n

#### Количество безработных, % 

In [33]:
job_df_mean = job_df.iloc[:, 1:5].mean(axis=1)
job_individ = 1000 * 100 * job_df_mean / population_mean 
job_individ_df = job_individ.to_frame(name='Количество безработных, %') 
job_individ_df = job_individ_df.reset_index()

### 1.6 Количество зданий организаций, осуществляющих образовательную деятельность по образовательным программам начального общего, основного общего, среднего общего образования (единица)

In [34]:
school = pd.read_excel('dop_social_data\school_building_2018.xls')

def std_data_school(df):
    '''Функция удаляет неинформативные столбцы и переименовывает их, удаляет первые две строки, 
    изменяет тип данных на числовой, сортирует регионы по алфавиту
    '''
    
    df = df.rename(columns={'Количество зданий организаций, осуществляющих образовательную деятельность по образовательным программам начального общего, основного общего, среднего общего образования (единица)': 'region', 
                            'Unnamed: 1': '2018'})    #переименовываем столбцы
    df = df.drop(index=[ 0 , 1, 2]) #удаляем строки  
    df.reset_index(drop= True , inplace= True )    #сброс индекса
    df.columns = df.columns.astype(object) 
    return df


school = pd.read_excel('dop_social_data\school_building_2018.xls')
school = std_data_school(school)
school = kredit_readout(school)

school['2018'] = school['2018'].astype(int)    #смена типа данных
school.columns = school.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
school.head()


,2018
region,
Алтайский край,1620
Амурская область,546
Архангельская область без Ненецкого АО,658
Астраханская область,548
Белгородская область,776


In [35]:
school.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2018    85 non-null     int32
dtypes: int32(1)
memory usage: 1020.0+ bytes


#### Количество школ на 1000 населения, шт.\
По состоянию на 2018 год.


In [36]:
school_individ = 1000*school['2018']/population['2018']
school_individ_df = school_individ.to_frame(name='Количество школ, шт./1000 чел.') 
school_individ_df = school_individ_df.reset_index()

### 1.7 Розничные продажи алкогольной продукции на душу населения (в литрах этанола)
https://www.fedstat.ru/indicator/59721

In [37]:
alko_cach = pd.read_excel('dop_social_data\data_alko_cach.xls')

def std_data_alko(df):
    '''Функция удаляет неинформативные столбцы и переименовывает их, удаляет первые две строки, 
    изменяет тип данных на числовой, сортирует регионы по алфавиту
    '''
    
    df = df.drop(['Unnamed: 1'], axis=1)  
    df = df.rename(columns={'Розничные продажи алкогольной продукции на душу населения (в литрах этанола)': 'region',
                            'Unnamed: 2': '2019',
                            'Unnamed: 3': '2020'
                            })    #переименовываем столбцы
    df = df.drop(index=[ 0 , 1, 2,3]) #удаляем строки  
    df.reset_index(drop= True , inplace= True )    #сброс индекса
    return df

alko_cach = std_data_alko(alko_cach)
alko_cach.columns = alko_cach.columns.astype(object)   
alko_cach = kredit_readout(alko_cach)

alko_cach['2019'] = alko_cach['2019'].astype(float)    #смена типа данных
alko_cach['2020'] = alko_cach['2020'].astype(float)    #смена типа данных
alko_cach.columns = alko_cach.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
alko_cach.head()

,2019,2020
region,,
Алтайский край,5.6216,5.3943
Амурская область,7.2232,7.7359
Архангельская область без Ненецкого АО,8.4267,8.7451
Астраханская область,4.4266,4.5826
Белгородская область,4.9824,4.8388


In [38]:
alko_cach.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    85 non-null     float64
 1   2020    85 non-null     float64
dtypes: float64(2)
memory usage: 2.0+ KB


#### Средний показатель за период

In [39]:
alko_cach_individ = alko_cach.mean(axis=1)
alko_cach_individ_df = alko_cach_individ.to_frame (name='Продажа алкоголя, л.')
alko_cach_individ_df = alko_cach_individ_df.reset_index()

### 1.8 Валовый региональный продукт на душу населения
Считываем данные с 2011 года - обеспечивается единая методика расчета по Архангельской и Тюменской областям.

In [40]:
grp = pd.read_excel('social_russia_data/gross_regional_product_1996_2020.xls', 
                    header=2, skipfooter=1, usecols='A, Q:Z')
grp.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']
grp = federal_subj(grp, standard_names, names_to_drop)
grp.columns = grp.columns.astype(object) 
grp.columns = grp.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
grp.head()

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,
Алтайский край,137677.2,153556.7,173763.5,186798.6,204933.1,224525.8,231268.4,247599.3,271319.7,291156.9
Амурская область,273175.8,280023.9,258817.0,286282.6,343385.7,370192.4,373935.1,419905.2,521060.1,571362.1
Архангельская область без Ненецкого АО,232540.7,270662.9,283264.5,310817.4,352837.9,400764.6,441961.6,493205.1,509917.0,514200.4
Астраханская область,170504.7,206677.1,269821.7,290822.2,315996.9,361704.8,434701.5,570206.4,596388.2,526950.9
Белгородская область,331010.0,354570.6,368874.8,400820.8,447619.7,501467.8,539720.5,588641.5,617426.5,646569.0


In [41]:
grp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2011    85 non-null     float64
 1   2012    85 non-null     float64
 2   2013    85 non-null     float64
 3   2014    85 non-null     float64
 4   2015    85 non-null     float64
 5   2016    85 non-null     float64
 6   2017    85 non-null     float64
 7   2018    85 non-null     float64
 8   2019    85 non-null     float64
 9   2020    85 non-null     float64
dtypes: float64(10)
memory usage: 7.3+ KB


#### Определяем средний показатель за период.

In [42]:
grp_mean = grp.iloc[:, 6:].mean(axis=1) 
grp_mean_df = grp_mean.to_frame(name='Валовый продукт, руб./чел.')
grp_mean_df = grp_mean_df.reset_index()

### 1.9 Характеристика жилищных условий домохозяйств
Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование 2020 года.

Выделяем:
- среднюю жилую площадь на одного человека;
- процент домохозяйств, указавших на плохое состояние жилого помещения;
- процент домохозяйств, не имеющих средств для улучшения жилищных условий (процент встающих в очередь на жилье или рассчитывающих на переселение в связи со сносом дома)

In [43]:
def housing_readout(file, sheet, cols, skipfooter=0):
    "Считывание данных с excel-листа"
    df = pd.read_excel(
        file, sheet_name=sheet, usecols=cols, skipfooter=skipfooter)
    df.rename({'Регион': 'region'}, axis=1, inplace=True)
    
    
    # Ошибки в написании названий (латиница?)
    df.replace({'Bладимирская область': 'Владимирская'}, inplace=True)
    df.replace({'Bолгоградская область': 'Волгоградская'}, inplace=True)
    df.replace({'Bологодская область': 'Вологодская'}, inplace=True)
    df.replace({'Bоронежская область': 'Воронежская'}, inplace=True)
    df.replace('...', 0, inplace=True)  # слабая статистика -> 0
    df = federal_subj(df, standard_names)
    df.columns = df.columns.astype(object) 
    return df


file = 'social_russia_data/housing_2020.xlsx'
living_area = housing_readout(file, 0, 'A, H')
living_area.columns.values[0] = 'living area'
intents = housing_readout(file, 1, 'A, C, E, H, I', 2)
intents.columns.values[0] = 'to improve'
intents.columns.values[1] = 'bad conditions'
intents.columns.values[2] = 'queue'
intents.columns.values[3] = 'demolition'
intents['no funds'] = ((intents['to improve']/100 
    * (intents['queue']/100 + intents['demolition']/100)) * 100)
housing = pd.concat(
    [living_area, intents[['bad conditions', 'no funds']]], axis=1)


Меняем название первой колонки

In [44]:
def std_func_hous(df):
    '''Функция переименовывает первый столбец.'''
    df = df.rename(columns={
        'region':'region',
        'living area':'Средняя жилая площадь на чел, м.кв.', 
        'bad conditions':'Ветхое жильё, м.кв.', 
        'no funds':'В очереди на переселение, чел.'
        })
    return df


housing = std_func_hous(housing)
#housing.columns = housing.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов

In [45]:
housing.head()

,"Средняя жилая площадь на чел, м.кв.","Ветхое жильё, м.кв.","В очереди на переселение, чел."
region,,,
Алтайский край,18.0,0.3,0.8642
Амурская область,17.3,1.3,1.0731
Архангельская область без Ненецкого АО,16.9,1.4,1.2998
Астраханская область,19.2,0.8,2.9028
Белгородская область,17.8,0.7,0.3840


In [46]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 3 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Средняя жилая площадь на чел, м.кв.  85 non-null     float64
 1   Ветхое жильё, м.кв.                  85 non-null     float64
 2   В очереди на переселение, чел.       85 non-null     float64
dtypes: float64(3)
memory usage: 2.7+ KB


#### Для представления в итоговом датасете возьмём отдельные столбцы

In [47]:
mean_squere = housing['Средняя жилая площадь на чел, м.кв.']
bad_squere = housing['Ветхое жильё, м.кв.']
stack_squere = housing['В очереди на переселение, чел.']

In [48]:
mean_squere_df = mean_squere.to_frame(name='Средняя жилая площадь на чел, м.кв.')
bad_squere_df = bad_squere.to_frame(name='Ветхое жильё, м.кв.')
stack_squere_df = stack_squere.to_frame(name='В очереди на переселение, чел.')

In [49]:
mean_squere_df = mean_squere_df.reset_index()
bad_squere_df = bad_squere_df.reset_index()
stack_squere_df = stack_squere_df.reset_index()

### 1.10 Заболеваемость на 100тыс. человек населения, по возрастным группам и группам заболеваний.

In [50]:
morbidity = pd.read_excel(
        'social_russia_data/morbidity_2005_2020_age_disease.xls',
        header=2, usecols='A:C, M:O')
morbidity.rename(
    {'Unnamed: 0': 'region', 'Unnamed: 1': 'type', 'Unnamed: 2': 'age'}, 
    axis=1, inplace=True)
morbidity = morbidity.dropna(subset=['2014', '2015', '2016'], how='all')
morbidity = federal_subj(morbidity, standard_names)

# Общая заболеваемость
mask = ((morbidity['type'] == 'Все заболевания') & (morbidity['age'] ==  'Всего'))
general_morbidity = morbidity[mask].copy()
general_morbidity.drop(columns=['type', 'age', '2014'], inplace=True)
general_morbidity.columns = general_morbidity.columns.astype(object) 

# Врожденные пороки развития
mask = (
    (morbidity['type'] == 'Врожденные аномалии (пороки развития), деформации и хромосомные нарушения')
    & (morbidity['age'] ==  'Всего')   
)
congenital_malformation = morbidity[mask].copy()
congenital_malformation.drop(columns=['type', 'age'], inplace=True)
congenital_malformation.columns = congenital_malformation.columns.astype(object) 

# Психические расстройства и расстройства поведения
mask = (
    (morbidity['type'] == 'Психические расстройства и расстройства поведения')
    & (morbidity['age'] ==  'Всего')   
)
psychosis = morbidity[mask].copy()
psychosis.drop(columns=['type', 'age'], inplace=True)
psychosis.columns = psychosis.columns.astype(object) 

# Подростковая беременность (до 14 лет)
mask = (
    (morbidity['type'] == 'Беременность, роды и послеродовой период')
    & (morbidity['age'] ==  '0-14 лет')   
)
teenage_pregnancy = morbidity[mask].copy()
teenage_pregnancy.drop(columns=['type', 'age'], inplace=True)
teenage_pregnancy.columns = teenage_pregnancy.columns.astype(object) 

general_morbidity.columns = general_morbidity.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
congenital_malformation.columns = congenital_malformation.columns.str.replace(' ', '')
psychosis.columns = psychosis.columns.str.replace(' ', '')
teenage_pregnancy.columns = teenage_pregnancy.columns.str.replace(' ', '')

display('Общая заболеваемость, на 100тыс.чел.', general_morbidity.head(3))
display('Врожденные пороки развития, на 100тыс.чел.', congenital_malformation.head(3))
display('Психические расстройства, на 100тыс.чел.', psychosis.head(3))
display('Подростковая беременность, на 100тыс.чел.', teenage_pregnancy.head(3))

'Общая заболеваемость, на 100тыс.чел.'

,2015,2016
region,,
Алтайский край,109652.6,112764.7
Амурская область,83919.0,83412.0
Архангельская область без Ненецкого АО,101587.4,99842.1


'Врожденные пороки развития, на 100тыс.чел.'

,2014,2015,2016
region,,,
Алтайский край,176.1,196.7,200.1
Амурская область,391.2,335.2,370.1
Архангельская область без Ненецкого АО,380.8,423.9,458.9


'Психические расстройства, на 100тыс.чел.'

,2014,2015,2016
region,,,
Алтайский край,1454.0,1467.7,1272.1
Амурская область,512.4,485.3,446.7
Архангельская область без Ненецкого АО,646.0,693.5,685.3


'Подростковая беременность, на 100тыс.чел.'

,2014,2015,2016
region,,,
Алтайский край,3.4,8.3,3.3
Амурская область,22.4,13.3,4.4
Архангельская область без Ненецкого АО,10.5,6.9,13.8


In [51]:
general_morbidity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2015    85 non-null     float64
 1   2016    85 non-null     float64
dtypes: float64(2)
memory usage: 2.0+ KB


#### Средние значения показателей  

In [52]:
mor_mean = general_morbidity.mean(axis=1)
cong_mean = congenital_malformation.mean(axis=1)
psycho_mean = psychosis.mean(axis=1)
teen_mean = teenage_pregnancy.mean(axis=1)

In [53]:
mor_mean_df = mor_mean.to_frame(name='Общая заболеваемость, на 100тыс.чел.')
cong_mean_df = cong_mean.to_frame(name='Врожденные пороки развития, на 100тыс.чел.')
psycho_mean_df = psycho_mean.to_frame(name='Психические расстройства, на 100тыс.чел.')
teen_mean_df = teen_mean.to_frame(name='Подростковая беременность, на 100тыс.чел.')

In [54]:
mor_mean_df = mor_mean_df.reset_index()
cong_mean_df = cong_mean_df.reset_index()
psycho_mean_df = psycho_mean_df.reset_index()
teen_mean_df = teen_mean_df.reset_index()

### 1.11 Число больничных организаций, на конец года

In [55]:
def std_med_org(df):
    '''Функция удаляет неинформативные столбцы и переименовывает их, удаляет первые две строки, 
    изменяет тип данных на числовой, сортирует регионы по алфавиту
    '''
    
    df = df.rename(columns={'Число больничных организаций, на конец года': 'region', 
                            'Unnamed: 1':'2004',
                            'Unnamed: 2':'2005',
                            'Unnamed: 3':'2006',
                            'Unnamed: 4':'2007',
                            'Unnamed: 5':'2008',
                            'Unnamed: 6':'2009',
                            'Unnamed: 7':'2010',
                            'Unnamed: 8':'2011',
                            'Unnamed: 9':'2012',
                            'Unnamed: 10':'2013',
                            'Unnamed: 11':'2014',
                            'Unnamed: 12':'2015',
                            'Unnamed: 13':'2016',
                            'Unnamed: 14':'2017',
                            'Unnamed: 15':'2018',
                            'Unnamed: 16':'2019',
                            'Unnamed: 17':'2020',
                            'Unnamed: 18':'2021',
                            })    #переименовываем столбцы
    df = df.drop(index=[ 0 , 1, 2]) #удаляем строки  
    df.columns = df.columns.astype(object) 
    df.reset_index(drop= True , inplace= True )    #сброс индекса
    return df


med_df = pd.read_excel('dop_social_data/сhislo_bolnich_organiz.xls')
med_df = std_med_org(med_df)
med_df = kredit_readout(med_df)
med_df.columns = med_df.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
med_df['2004'] = med_df['2004'].astype(float)
med_df.head()

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,
Алтайский край,248.0,255.0,250.0,152.0,140.0,140.0,142.0,139.0,134.0,136.0,131.0,132.0,134.0,129.0,129.0,129.0,125.0,124.0
Амурская область,114.0,113.0,89.0,82.0,68.0,67.0,68.0,70.0,51.0,42.0,42.0,42.0,38.0,38.0,40.0,41.0,39.0,42.0
Архангельская область без Ненецкого АО,117.0,89.0,83.0,83.0,79.0,75.0,71.0,62.0,61.0,62.0,65.0,61.0,63.0,63.0,59.0,58.0,58.0,47.0
Астраханская область,73.0,71.0,47.0,48.0,43.0,43.0,44.0,45.0,41.0,39.0,39.0,37.0,37.0,35.0,34.0,34.0,34.0,33.0
Белгородская область,95.0,87.0,60.0,60.0,59.0,56.0,56.0,54.0,53.0,53.0,53.0,52.0,50.0,50.0,43.0,42.0,42.0,46.0


In [56]:
med_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2004    82 non-null     float64
 1   2005    83 non-null     float64
 2   2006    83 non-null     float64
 3   2007    83 non-null     float64
 4   2008    83 non-null     float64
 5   2009    83 non-null     float64
 6   2010    83 non-null     float64
 7   2011    83 non-null     float64
 8   2012    83 non-null     float64
 9   2013    83 non-null     float64
 10  2014    85 non-null     float64
 11  2015    85 non-null     float64
 12  2016    85 non-null     float64
 13  2017    85 non-null     float64
 14  2018    85 non-null     float64
 15  2019    85 non-null     float64
 16  2020    85 non-null     float64
 17  2021    85 non-null     float64
dtypes: float64(18)
memory usage: 12.6+ KB


#### Рассчитаем показатель на одну тысячу населения.

In [57]:
med_mean = med_df.iloc[:, 13:17].mean(axis=1) 
med_org = 1000 * med_mean / population_mean
med_org_df = med_org.to_frame (name='Число больниц, ед.')
med_org_df = med_org_df.reset_index()

### 1.12 Новорождённые по регионам, без учёта мертворождённых.

In [58]:
newborn = pd.read_csv(
    'social_russia_data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')
newborn.drop(columns='Unnamed: 198',inplace=True)
newborn.rename({'Region': 'region'}, axis=1, inplace=True)
names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]
newborn = federal_subj(newborn, standard_names, names_to_drop=names_to_drop)
newborn.loc['Архангельская область без Ненецкого АО'] \
    = (newborn.loc['Архангельская область без Ненецкого АО']
      - newborn.loc['Ненецкий автономный округ'])
newborn.loc['Тюменская область без округов'] \
    = (newborn.loc['Тюменская область без округов']
       - newborn.loc['Ханты-Мансийский автономный округ - Югра']
       - newborn.loc['Ямало-Ненецкий автономный округ'])
    
# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
newborn.drop(columns=2022, inplace=True)  # Неполный год
newborn = newborn.round(0).astype(int)
newborn.columns = newborn.columns.astype(object) 
#newborn.columns = newborn.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов

newborn.head(3)

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,
Алтайский край,25513,28209,30711,31038,30863,30739,32695,32461,31429,30065,28867,25525,23491,21117,19976,18992
Амурская область,10405,10935,11233,11384,11473,11188,11733,11483,11159,10781,10377,9430,8889,7919,7853,7343
Архангельская область без Ненецкого АО,13396,14438,14577,14776,14806,14253,14730,14584,14331,13908,13313,11714,10671,9662,8906,8487


In [59]:
newborn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2006    85 non-null     int32
 1   2007    85 non-null     int32
 2   2008    85 non-null     int32
 3   2009    85 non-null     int32
 4   2010    85 non-null     int32
 5   2011    85 non-null     int32
 6   2012    85 non-null     int32
 7   2013    85 non-null     int32
 8   2014    85 non-null     int32
 9   2015    85 non-null     int32
 10  2016    85 non-null     int32
 11  2017    85 non-null     int32
 12  2018    85 non-null     int32
 13  2019    85 non-null     int32
 14  2020    85 non-null     int32
 15  2021    85 non-null     int32
dtypes: int32(16)
memory usage: 8.0+ KB


#### Рассчитаем показатель на одну тысячу населения

In [60]:
newborn_mean = newborn.iloc[:, 11:15].mean(axis=1) 
newborn_individ = 1000 * newborn_mean / population_mean
newborn_individ_df = newborn_individ.to_frame (name='Рождаемость, чел./1000 чел.')
newborn_individ_df = newborn_individ_df.reset_index()

### 1.13 Количество населения за чертой бедности (доходы ниже прожиточного минимума), %

In [61]:
poverty_percent = pd.read_csv('social_russia_data/poverty_percent_by_regions_1992_2020.csv')
poverty_percent = pd.pivot_table(
    data=poverty_percent, 
    values='poverty_percent',
    columns='year', 
    index='region'
    ).reset_index()
names_to_drop = ['Архангельская область', 'Тюменская область']
poverty_percent = federal_subj(poverty_percent, standard_names, names_to_drop)
poverty_percent.columns = pd.to_datetime(poverty_percent.columns, format='%Y').year
poverty_percent.head(3)

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,NaN,NaN,NaN,33.7,46.8,45.7,52.9,53.8,53.9,47.3,...,22.6,20.6,17.6,17.1,18.0,17.8,17.5,17.4,17.6,17.5
Амурская область,NaN,NaN,NaN,36.1,28.2,26.3,31.2,38.0,47.7,45.3,...,20.4,16.0,16.2,14.8,15.2,17.0,16.7,15.6,15.7,15.2
Архангельская область без Ненецкого АО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3


In [62]:
poverty_percent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, Алтайский край to Ярославская область
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1992    0 non-null      float64
 1   1993    0 non-null      float64
 2   1994    0 non-null      float64
 3   1995    82 non-null     float64
 4   1996    82 non-null     float64
 5   1997    82 non-null     float64
 6   1998    82 non-null     float64
 7   1999    82 non-null     float64
 8   2000    82 non-null     float64
 9   2001    82 non-null     float64
 10  2002    82 non-null     float64
 11  2003    83 non-null     float64
 12  2004    83 non-null     float64
 13  2005    83 non-null     float64
 14  2006    83 non-null     float64
 15  2007    83 non-null     float64
 16  2008    83 non-null     float64
 17  2009    83 non-null     float64
 18  2010    83 non-null     float64
 19  2011    83 non-null     float64
 20  2012    85 non-null     float64
 21  2013    85 non-n

#### Преобразуем в среднее за рассматриваемый период

In [63]:
poverty_mean = poverty_percent.iloc[:, 25:].mean(axis=1) 
poverty_mean_df = poverty_mean.to_frame (name='Бедность, %')
poverty_mean_df = poverty_mean_df.reset_index()

### 1.14 Распределение малоимущего населения по социально-демографическим группам

In [64]:
def socdem_readout(file):
    df = pd.read_excel(file, header=2)
    df.rename({
        'Unnamed: 0': 'region',
        'Все население': 'all',
        'Дети в возрасте до 16 лет': 'children',
        'Население старше трудоспособного возраста': 'retired',
        'Население трудоспособного возраста': 'employable'
        }, axis=1, inplace=True)
    
    names_to_drop = [
        '        Архангельская область',
        '        Тюменская область'
    ]
    df = federal_subj(df, standard_names, names_to_drop)
    return df


poverty_socdem_2017 = socdem_readout('social_russia_data/poverty_socdem_2017.xls')
poverty_socdem_2018 = socdem_readout('social_russia_data/poverty_socdem_2018.xls')
poverty_socdem_2019 = socdem_readout('social_russia_data/poverty_socdem_2019.xls')
poverty_socdem_2020 = socdem_readout('social_russia_data/poverty_socdem_2020.xls')

poverty_socdem = poverty_socdem_2020.copy()

mask = (poverty_socdem[['children', 'retired', 'employable']].isna().all(axis=1))
poverty_socdem[mask] = poverty_socdem_2019

mask = ((poverty_socdem[['children', 'retired', 'employable']] == 0).any(axis=1))
poverty_socdem[mask] = poverty_socdem_2018

mask = ((poverty_socdem[['children', 'retired', 'employable']] == 0).any(axis=1))
poverty_socdem[mask] = poverty_socdem_2017

poverty_socdem.columns = poverty_socdem.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов


In [65]:
poverty_socdem.head(3)

,all,children,retired,employable
region,,,,
Алтайский край,100,31.4,9.2,59.4
Амурская область,100,38.4,4.7,56.9
Архангельская область без Ненецкого АО,100,28.5,7.2,64.3


In [66]:
poverty_socdem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   all         85 non-null     int64  
 1   children    85 non-null     float64
 2   retired     85 non-null     float64
 3   employable  85 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 3.3+ KB


#### Отберём отдельные столбцы по категориям

In [67]:
children = poverty_socdem['children']
retired = poverty_socdem['retired']
employable = poverty_socdem['employable']

In [68]:
children_df = children.to_frame (name='Дети до 16 лет, чел.')
retired_df = retired.to_frame (name='Пенсионеры,чел.')
employable_df = employable.to_frame (name='Трудящиеся, чел.')

In [69]:
children_df = children_df.reset_index()
retired_df = retired_df.reset_index()
employable_df = employable_df.reset_index()

### 1.15 Региональное производство
Объем отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности, в тысячах рублей

In [70]:
reg_prod = pd.read_csv('social_russia_data/regional_production_2017_2020.csv')
names_to_drop = ['Архангельская область', 'Тюменская область']
reg_prod = federal_subj(reg_prod, standard_names, names_to_drop) 
reg_prod.columns.values[-4:] = pd.to_datetime(reg_prod.columns[-4:]).year
reg_prod_total = (reg_prod.groupby('region')[[2017, 2018, 2019, 2020]].sum()) 

reg_prod.columns = reg_prod.columns.astype(object) 
reg_prod_total.columns = reg_prod_total.columns.astype(object) 

display('Региональное производство, по видам деятельности', reg_prod.head(5))
display('Региональное производство, общий объем', reg_prod_total.head(3))

'Региональное производство, по видам деятельности'

,production_field,2017,2018,2019,2020
region,,,,,
Алтайский край,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,6143029.4,5.023392e+06,7745963.3,9.146146e+06
Алтайский край,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПА...",45806615.7,4.743969e+07,43900541.6,4.474102e+07
Алтайский край,ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,307969929.4,3.228103e+08,357699013.3,3.618801e+08
Алтайский край,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОР...",8068621.4,1.032303e+07,9813456.6,1.045441e+07
Алтайский край,Промышленное производство (промышленность),367988195.7,3.855964e+08,419158974.7,4.262217e+08


'Региональное производство, общий объем'

,2017,2018,2019,2020
region,,,,
Алтайский край,7.359764e+08,7.711927e+08,8.383179e+08,8.524434e+08
Амурская область,2.688441e+08,2.687608e+08,3.400267e+08,4.045765e+08
Архангельская область без Ненецкого АО,2.005225e+09,1.954098e+09,1.798199e+09,1.854125e+09


In [71]:
reg_prod_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2017    85 non-null     float64
 1   2018    85 non-null     float64
 2   2019    85 non-null     float64
 3   2020    85 non-null     float64
dtypes: float64(4)
memory usage: 3.3+ KB


#### Для признака будет расчитано среднее значение на тысячу человек населения, проживающих в регионе

In [72]:
reg_prod_total_mean = reg_prod_total.mean(axis=1) 
reg_prod_individ = 1000 * reg_prod_total_mean / population_mean
reg_prod_individ_df = reg_prod_individ.to_frame (name='Региональный продукт, на тыс. чел.')
reg_prod_individ_df.reset_index()

,region,"Региональный продукт, на тыс. чел."
0,Алтайский край,3.414504e+05
1,Амурская область,4.027777e+05
2,Архангельская область без Ненецкого АО,1.719929e+06
3,Астраханская область,7.732429e+05
4,Белгородская область,1.158820e+06
...,...,...
80,Чеченская Республика,5.086143e+04
81,Чувашская Республика,3.873585e+05
82,Чукотский автономный округ,3.766386e+06
83,Ямало-Ненецкий автономный округ,1.031160e+07


### 1.16 Розничная торговля на душу населения

In [73]:
retail = pd.read_excel(
    'social_russia_data/retail_turnover_per_capita_2000_2021.xls', 
    header=2, usecols='A, P:X')
retail.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
retail.drop(0, inplace=True)
names_to_drop = [
        '        Архангельская область',
        '            Архангельская область (без АО)',
        '        Тюменская область',
        '            Тюменская область (без АО)'
]
retail = federal_subj(retail, standard_names, names_to_drop)
retail.columns = retail.columns.astype(object) 
retail.columns = retail.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
retail.head(3)

,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Алтайский край,118096,128376,134925,137844,143873,150444,159514,153605,174321
Амурская область,145301,163781,182491,191523,202038,214688,231113,245233,276635
Архангельская область без Ненецкого АО,154177,176420,194345,202977,217332,229922,240155,250033,280050


In [74]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   2013    83 non-null     object
 1   2014    85 non-null     object
 2   2015    85 non-null     object
 3   2016    85 non-null     object
 4   2017    85 non-null     object
 5   2018    85 non-null     object
 6   2019    85 non-null     object
 7   2020    85 non-null     object
 8   2021    85 non-null     object
dtypes: object(9)
memory usage: 6.6+ KB


#### Определим средний показатель за исследуемый период

In [75]:
retail_mean = retail.iloc[:, 4:8].mean(axis=1)
retail_mean_df = retail_mean.to_frame (name='Розничная торговля на душу населения, руб./чел.')
retail_mean_df = retail_mean_df.reset_index()

### 1.17 Общий миграционный прирост
Будут рассматриваться данные за 2017-2020 годы. Информация содержится в ежегодных отчётах на отдельных листах. Выделим только показатель количества мигрантов за этот период.

Исследуем даннные за 2017 год.

In [76]:
migr_df_17 = pd.read_excel('dop_social_data\migr17.xlsx', sheet_name='Таб 2-02 МигрПрирост')


def std_data_alko(df):
    '''Функция удаляет неинформативные столбцы и переименовывает их, удаляет первые две строки, 
    изменяет тип данных на числовой, сортирует регионы по алфавиту
    '''
    
    df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)  
    df = df.rename(columns={'Unnamed: 0': 'region', 'Unnamed: 1': 'Мигр.Прирост'})    #переименовываем столбцы
    df = df.drop(index=[ 0 , 1, 2, 3]) #удаляем строки  
    df.reset_index(drop= True , inplace= True )    #сброс индекса
    return df


migr_df_17 = std_data_alko(migr_df_17)
migr_df_17 = kredit_readout(migr_df_17)

migr_df_17['Мигр.Прирост'] = migr_df_17['Мигр.Прирост'].astype(int)    #смена типа данных
migr_df_17.columns = migr_df_17.columns.astype(object) 
migr_df_17.head()

,Мигр.Прирост
region,
Алтайский край,-8059
Амурская область,-2104
Архангельская область без Ненецкого АО,-7814
Астраханская область,-2002
Белгородская область,2839


In [77]:
migr_df_17.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Мигр.Прирост  85 non-null     int32
dtypes: int32(1)
memory usage: 1020.0+ bytes


Исследуем даннные за 2018-2020 годы.

In [78]:
def migr_readout(df):
    '''Общая функция для чтения и стандартизации отчётов о миграционном приросте за 2018-2020 годы.'''
        
    def std_data_alko(df):
        '''Функция удаляет неинформативные столбцы и переименовывает их, удаляет первые две строки, 
        изменяет тип данных на числовой, сортирует регионы по алфавиту
        '''
        
        df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', ], axis=1)  
        df = df.rename(columns={'Содержание': 'region', 'Unnamed: 1': 'Мигр.Прирост'})    #переименовываем столбцы
        df = df.drop(index=[ 0 , 1, 2, 3]) #удаляем строки  
        df.reset_index(drop= True , inplace= True )    #сброс индекса
        df.columns = df.columns.astype(object) 
        return df

    df = std_data_alko(df)
    df = kredit_readout(df)
    #df = reset_index_df(df)
    df['Мигр.Прирост'] = df['Мигр.Прирост'].astype(int)    #смена типа данных
    df.head()
    return df


migr_df_18 = pd.read_excel('dop_social_data\migr18.xlsx', sheet_name='2.2_МигрПр')
migr_df_18 = migr_readout(migr_df_18)
migr_df_19 = pd.read_excel('dop_social_data\migr19.xlsx', sheet_name='2.2_МигрПр')
migr_df_19 = migr_readout(migr_df_19)
migr_df_20 = pd.read_excel('dop_social_data\migr20.xlsx', sheet_name='2.2_МигрПр')
migr_df_20 = migr_readout(migr_df_20)
migr_df_20.head()

,Мигр.Прирост
region,
Алтайский край,-2416
Амурская область,-3144
Архангельская область без Ненецкого АО,-2183
Астраханская область,-5123
Белгородская область,4229


In [79]:
migr_df_20.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Мигр.Прирост  85 non-null     int32
dtypes: int32(1)
memory usage: 1020.0+ bytes


In [80]:
migr_df_17 = migr_df_17.rename(columns={'Мигр.Прирост': '2017'})
migr_df_18 = migr_df_18.rename(columns={'Мигр.Прирост': '2018'})
migr_df_19 = migr_df_19.rename(columns={'Мигр.Прирост': '2019'})
migr_df_20 = migr_df_20.rename(columns={'Мигр.Прирост': '2020'})

migr_joined = pd.concat([migr_df_17, migr_df_18, migr_df_19, migr_df_20], axis=1)
migr_joined.columns = migr_joined.columns.str.replace(' ', '') #Убираем пробелы в названиях столбцов
migr_joined.head()

,2017,2018,2019,2020
region,,,,
Алтайский край,-8059,-7411,-4151,-2416
Амурская область,-2104,-3392,12,-3144
Архангельская область без Ненецкого АО,-7814,-6701,-3069,-2183
Астраханская область,-2002,-3496,-7838,-5123
Белгородская область,2839,4123,9212,4229


In [81]:
migr_joined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2017    85 non-null     int32
 1   2018    85 non-null     int32
 2   2019    85 non-null     int32
 3   2020    85 non-null     int32
dtypes: int32(4)
memory usage: 2.0+ KB


#### Рассчитаем прирост миграции по отношению к численности населения

In [82]:
migr_joined_mean = migr_joined.mean(axis=1) 
migr_individ = 1000 * migr_joined_mean / population_mean
migr_individ_df = migr_individ.to_frame (name='Миграционный прирост, на тыс.чел')
migr_individ_df = migr_individ_df.reset_index()

### 1.18 Сведения о заболеваемости алкоголизмом и наркоманией, на 1 тыс. населения
Данные получены из расчёта на сто тысяч, но будут приведены к расчёту на одну.

In [83]:
def alco_readout(file, sheet):
    '''Считывание данных с excel-листа'''
    df = pd.read_excel(file, sheet_name=sheet)
    df = federal_subj(df, standard_names)
    df.columns = df.columns.astype(object) 
    return df


file = 'social_russia_data/drug_alco.xlsx'
alco = alco_readout(file, 0)
alco_1 = alco_readout(file, 1)
drug = alco_readout(file, 2)
drug_1 = alco_readout(file, 3)
alco = pd.concat([alco, alco_1], axis=1).sort_index()
drug = pd.concat([drug, drug_1], axis=1).sort_index()

display('Алкоголизм, чел. на 1000 населения', alco.head())
display('Наркомания, чел. на 1000 населения', drug.head())

'Алкоголизм, чел. на 1000 населения'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область без Ненецкого АО,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7
Астраханская область,137.1,166.0,130.3,116.1,134.5,113.4,91.7,84.7,61.8,NaN,NaN,NaN,26.2,22.2
Белгородская область,99.6,93.2,84.0,92.3,94.3,86.4,72.6,74.8,66.9,56.8,50.7,46.1,45.2,41.3


'Наркомания, чел. на 1000 населения'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область без Ненецкого АО,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2
Астраханская область,10.7,12.9,12.9,6.8,8.1,9.8,7.5,5.1,2.3,NaN,NaN,NaN,0.7,0.2
Белгородская область,4.5,9.5,9.5,4.2,3.9,4.8,4.0,2.3,2.4,1.9,2.0,1.6,2.3,3.2


In [84]:
alco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2005    83 non-null     float64
 1   2006    83 non-null     float64
 2   2007    83 non-null     float64
 3   2008    83 non-null     float64
 4   2009    83 non-null     float64
 5   2010    83 non-null     float64
 6   2011    83 non-null     float64
 7   2012    83 non-null     float64
 8   2013    83 non-null     float64
 9   2014    68 non-null     float64
 10  2015    68 non-null     float64
 11  2016    68 non-null     float64
 12  2017    85 non-null     object 
 13  2018    85 non-null     object 
dtypes: float64(12), object(2)
memory usage: 10.0+ KB


#### Усреднённые значения за 2017-2018 года

In [85]:
alco_mean = alco.iloc[:, 12:].mean(axis=1) 
alco_mean_df = alco_mean.to_frame (name='Алкоголизм, на тыс.чел')
alco_mean_df.reset_index()

,region,"Алкоголизм, на тыс.чел"
0,Алтайский край,87.00
1,Амурская область,102.35
2,Архангельская область без Ненецкого АО,60.00
3,Астраханская область,24.20
4,Белгородская область,43.25
...,...,...
80,Чеченская Республика,3.55
81,Чувашская Республика,77.70
82,Чукотский автономный округ,231.40
83,Ямало-Ненецкий автономный округ,93.30


In [86]:
drug_mean = drug.iloc[:, 12:].mean(axis=1) 
drug_mean_df = drug_mean.to_frame (name='Наркомания, на тыс.чел')
drug_mean_df = drug_mean_df.reset_index()

## ⭐️ 2. Объединение таблиц



In [ ]:
regions_df.to_csv('social_data/regions_df.csv')